## Flavor transition probabilities

This notebook produces splines for $P( \hat{\nu_s} \to \nu_e)$ in the energy region of interest. 

It also plots several cases of interest.

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib 
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
from matplotlib.pyplot import *
from matplotlib.legend_handler import HandlerLine2D

from source import flavour_transitions as osc
from source.const import *

### Generating oscillation probability spline

In [32]:
from scipy.interpolate import RegularGridInterpolator

In [67]:
NPOINTS=100
P_nu=np.zeros((NPOINTS,NPOINTS,NPOINTS))
P_nubar=np.zeros((NPOINTS,NPOINTS,NPOINTS))
P_nu_app=np.zeros((NPOINTS,NPOINTS,NPOINTS))
P_nubar_app=np.zeros((NPOINTS,NPOINTS,NPOINTS))
xue = np.logspace(-3.1,-0.9,NPOINTS)
xumu = np.linspace(0,0.1,NPOINTS)
en = np.logspace(-2,np.log10(20),NPOINTS)
for i in range(NPOINTS):
    for j in range(NPOINTS):
            USQR = [xue[i], xumu[j], 0.0]
            P_nu[i,j,:] = osc.Pse_avg(en, nue_to_nue, USQR=USQR, approx=False)
            P_nubar[i,j,:] = osc.Pse_avg(en, -nue_to_nue, USQR=USQR, approx=False)
            P_nu_app[i,j,:] = osc.Pse_avg(en, nue_to_nue, USQR=USQR, approx=True)
            P_nubar_app[i,j,:] = osc.Pse_avg(en, -nue_to_nue, USQR=USQR, approx=True)

In [69]:
func_nu = RegularGridInterpolator((xue, xumu, en), P_nu, method='linear',bounds_error=False,fill_value=0.0)
np.save('prob_splines/Pse_nu.npy',{"interp": func_nu})
func_nu_approx = RegularGridInterpolator((xue, xumu, en), P_nu_app, method='linear',bounds_error=False,fill_value=0.0)
np.save('prob_splines/Pse_nu_approx.npy',{"interp": func_nu_approx})

func_nubar = RegularGridInterpolator((xue, xumu, en), P_nubar, method='linear',bounds_error=False,fill_value=0.0)
np.save('prob_splines/Pse_nubar.npy',{"interp": func_nubar})
func_nubar_approx = RegularGridInterpolator((xue, xumu, en), P_nubar_app, method='linear',bounds_error=False,fill_value=0.0)
np.save('prob_splines/Pse_nubar_approx.npy',{"interp": func_nubar_approx})

load = np.load('prob_splines/Pse_nubar.npy',allow_pickle=True)
func_nubar = load.item()['interp']

Enu  =  np.logspace(-2,np.log10(20),100)
plt.plot(Enu, func_nubar(np.array([0.01*Enu/Enu,0.001*Enu/Enu,Enu]).T))

### Plotting probabilities 

In [18]:
def plot_for_USQR(uesqr=0.01, umusqr=0.1, utausqr=0.00):
    fsize=14
    rc('text', usetex=True)
    rcparams={'axes.labelsize':9,'xtick.labelsize':fsize,'ytick.labelsize':fsize,\
                    'figure.figsize':(1.2*3.7,6)	}
    rc('font',**{'family':'serif', 'serif': ['computer modern roman']})
    matplotlib.rcParams['hatch.linewidth'] = 0.1  # previous pdf hatch linewidth

    rcParams.update(rcparams)
    axes_form1  = [0.16,0.13+0.46,0.82,0.35]
    axes_form2  = [0.16,0.11,0.82,0.35]
    fig = plt.figure()
    ax = fig.add_axes(axes_form1)
    ax2 = fig.add_axes(axes_form2)


    E = np.logspace(np.log10(0.1),np.log10(50),100)

    USQR = [uesqr, umusqr, utausqr]
    # neutrinos
    ax.text(30,1.05,r'$\nu$',fontsize=15)
    ax.set_title(fr'$|U_{{e4}}|^2={uesqr}, |U_{{\mu4}}|^2={umusqr}, |U_{{\tau4}}|^2={utausqr}$', fontsize=14)
    ax.plot(E, osc.Pse_avg(E, nue_to_nue, USQR=USQR, approx=False), lw=1.0, color='black', dashes=(6,0), label=r'$\left\langle P_{\hat{\nu}_{s}\to \nu_e}\right\rangle_N$')
    us1, us2, us3 = osc.Usi_matter_avg(E, nue_to_nue, USQR=USQR, approx=False)
    ax.plot(E, us1, lw=1.5, color='violet', dashes=(2,1), label=r'$\left\langle |U_{s1}^m|^2\right\rangle_N$')
    ax.plot(E, us2, lw=1.5, color='dodgerblue', dashes=(4,1), label=r'$\left\langle |U_{s2}^m|^2\right\rangle_N$')
    ax.plot(E, us3, lw=1.5, color='darkorange', dashes=(6,3), label=r'$\left\langle|U_{s3}^m|^2\right\rangle_N$')

    # antineutrinos
    ax2.text(30,1.05,r'$\overline{\nu}$',fontsize=15)
    ax2.plot(E, osc.Pse_avg(E, -nue_to_nue, USQR=USQR, approx=False), lw=1.0, color='black', dashes=(6,0), label=r'$\left\langle P_{\overline{\hat{\nu}_{s}}\to \overline{\nu_e}}\right\rangle_N$')
    us1, us2, us3 = osc.Usi_matter_avg(E, -nue_to_nue, USQR=USQR, approx=False)
    ax2.plot(E, us1, lw=1.5, color='violet', dashes=(2,1), label=r'$\left\langle |U_{s1}^m|^2\right\rangle_N$')
    ax2.plot(E, us2, lw=1.5, color='dodgerblue', dashes=(4,1), label=r'$\left\langle |U_{s2}^m|^2\right\rangle_N$')
    ax2.plot(E, us3, lw=1.5, color='darkorange', dashes=(6,3), label=r'$\left\langle |U_{s3}^m|^2\right\rangle_N$')


    ##############
    # STYLE
    # ax.set_yscale('log')
    ax.legend(loc='upper left',frameon=False,ncol=2,fontsize=12,handletextpad=0.3)
    ax2.legend(loc='upper left',frameon=False,ncol=2,fontsize=12,handletextpad=0.3)

    ax.set_ylim(0,1.2)
    ax2.set_ylim(0,1.2)

    ax.set_xlim(0.1,50)
    ax2.set_xlim(0.1,50)

    ax.set_xscale('log')
    ax2.set_xscale('log')

    ax.set_ylabel(r'Probability',fontsize=15)
    ax2.set_ylabel(r'Probability',fontsize=15)

    ax.set_xlabel(r'$E_\nu/$MeV',fontsize=15)
    ax2.set_xlabel(r'$E_\nu/$MeV',fontsize=15)

    fig.savefig(f'plots/Psolar_usqr_{uesqr:.1g}_{umusqr:.1g}_{utausqr:.1g}.pdf')
    
    

In [73]:
plot_for_USQR(uesqr=0.01, umusqr=0.00, utausqr=0.00)
plot_for_USQR(uesqr=0.01, umusqr=0.001, utausqr=0.00)
plot_for_USQR(uesqr=0.01, umusqr=0.01, utausqr=0.00)
plot_for_USQR(uesqr=0.01, umusqr=0.10, utausqr=0.00)

plot_for_USQR(uesqr=0.1, umusqr=0.00, utausqr=0.00)
plot_for_USQR(uesqr=0.1, umusqr=0.01, utausqr=0.00)
plot_for_USQR(uesqr=0.1, umusqr=0.10, utausqr=0.00)

/Users/matheushostert/Repos/solar-neutrino-visible-decays/source/flavour_transitions.py:36: RuntimeWarning: invalid value encountered in true_divide
  rx = ma.masked_array(data=-VNC/VCC, mask= ~(VCC != 0), fill_value=0.0)
/Users/matheushostert/Repos/solar-neutrino-visible-decays/source/flavour_transitions.py:36: RuntimeWarning: invalid value encountered in true_divide
  rx = ma.masked_array(data=-VNC/VCC, mask= ~(VCC != 0), fill_value=0.0)
/Users/matheushostert/Repos/solar-neutrino-visible-decays/source/flavour_transitions.py:36: RuntimeWarning: invalid value encountered in true_divide
  rx = ma.masked_array(data=-VNC/VCC, mask= ~(VCC != 0), fill_value=0.0)
/Users/matheushostert/Repos/solar-neutrino-visible-decays/source/flavour_transitions.py:36: RuntimeWarning: invalid value encountered in true_divide
  rx = ma.masked_array(data=-VNC/VCC, mask= ~(VCC != 0), fill_value=0.0)
/Users/matheushostert/Repos/solar-neutrino-visible-decays/source/flavour_transitions.py:36: RuntimeWarning: inva

In [23]:
def plot_for_USQR_compare_approx(uesqr=0.01, umusqr=0.1, utausqr=0.00):
    fsize=14
    rc('text', usetex=True)
    rcparams={'axes.labelsize':9,'xtick.labelsize':fsize,'ytick.labelsize':fsize,\
                    'figure.figsize':(1.2*3.7,6)	}
    rc('font',**{'family':'serif', 'serif': ['computer modern roman']})
    matplotlib.rcParams['hatch.linewidth'] = 0.1  # previous pdf hatch linewidth

    rcParams.update(rcparams)
    axes_form1  = [0.16,0.13+0.46,0.82,0.35]
    axes_form2  = [0.16,0.11,0.82,0.35]
    fig = plt.figure()
    ax = fig.add_axes(axes_form1)
    ax2 = fig.add_axes(axes_form2)


    E = np.logspace(np.log10(0.1),np.log10(50),100)

    USQR = [uesqr, umusqr, utausqr]
    # neutrinos
    ax.text(30,1.05,r'$\nu$',fontsize=15)
    ax.set_title(fr'$|U_{{e4}}|^2={uesqr}, |U_{{\mu4}}|^2={umusqr}, |U_{{\tau4}}|^2={utausqr}$', fontsize=14)
    ax.plot(E, osc.Pse_avg(E, nue_to_nue, USQR=USQR, approx=False), lw=1.0, color='black', dashes=(6,0), label=r'$\left\langle P_{\hat{\nu}_{s}\to \nu_e}\right\rangle_N$')
    ax.plot(E, osc.Pse_avg(E, nue_to_nue, USQR=USQR, approx=True), lw=1.0, color='red', dashes=(6,0), label=r'$\left\langle P_{\hat{\nu}_{s}\to \nu_e}\right\rangle_N$ approx')
  
    # antineutrinos
    ax2.text(30,1.05,r'$\overline{\nu}$',fontsize=15)
    ax2.plot(E, osc.Pse_avg(E, -nue_to_nue, USQR=USQR, approx=False), lw=1.0, color='black', dashes=(6,0), label=r'$\left\langle P_{\overline{\hat{\nu}_{s}}\to \overline{\nu_e}}\right\rangle_N$')
    ax2.plot(E, osc.Pse_avg(E, -nue_to_nue, USQR=USQR, approx=True), lw=1.0, color='red', dashes=(6,0), label=r'$\left\langle P_{\overline{\hat{\nu}_{s}}\to \overline{\nu_e}}\right\rangle_N$ approx')
 
    ##############
    # STYLE
    # ax.set_yscale('log')
    ax.legend(loc='upper left',frameon=False,ncol=2,fontsize=12,handletextpad=0.3)
    ax2.legend(loc='upper left',frameon=False,ncol=2,fontsize=12,handletextpad=0.3)

    ax.set_ylim(0,1.2)
    ax2.set_ylim(0,1.2)

    ax.set_xlim(0.1,50)
    ax2.set_xlim(0.1,50)

    ax.set_xscale('log')
    ax2.set_xscale('log')

    ax.set_ylabel(r'Probability',fontsize=15)
    ax2.set_ylabel(r'Probability',fontsize=15)

    ax.set_xlabel(r'$E_\nu/$MeV',fontsize=15)
    ax2.set_xlabel(r'$E_\nu/$MeV',fontsize=15)

    fig.savefig(f'plots/Psolar_approx_usqr_{uesqr:.1g}_{umusqr:.1g}_{utausqr:.1g}.pdf')

In [29]:
plot_for_USQR_compare_approx(uesqr=0.01, umusqr=0.00, utausqr=0.00)
plot_for_USQR_compare_approx(uesqr=0.01, umusqr=0.01, utausqr=0.00)
plot_for_USQR_compare_approx(uesqr=0.01, umusqr=0.10, utausqr=0.00)

plot_for_USQR_compare_approx(uesqr=0.1, umusqr=0.00, utausqr=0.00)
plot_for_USQR_compare_approx(uesqr=0.1, umusqr=0.01, utausqr=0.00)
plot_for_USQR_compare_approx(uesqr=0.1, umusqr=0.10, utausqr=0.00)

plot_for_USQR_compare_approx(uesqr=0.01, umusqr=0.00, utausqr=0.10)

/Users/matheushostert/Repos/solar-neutrino-visible-decays/source/flavour_transitions.py:36: RuntimeWarning: invalid value encountered in true_divide
  rx = ma.masked_array(data=-VNC/VCC, mask= ~(VCC != 0), fill_value=0.0)
/Users/matheushostert/Repos/solar-neutrino-visible-decays/source/flavour_transitions.py:36: RuntimeWarning: invalid value encountered in true_divide
  rx = ma.masked_array(data=-VNC/VCC, mask= ~(VCC != 0), fill_value=0.0)
/Users/matheushostert/Repos/solar-neutrino-visible-decays/source/flavour_transitions.py:36: RuntimeWarning: invalid value encountered in true_divide
  rx = ma.masked_array(data=-VNC/VCC, mask= ~(VCC != 0), fill_value=0.0)
/Users/matheushostert/Repos/solar-neutrino-visible-decays/source/flavour_transitions.py:36: RuntimeWarning: invalid value encountered in true_divide
  rx = ma.masked_array(data=-VNC/VCC, mask= ~(VCC != 0), fill_value=0.0)
/Users/matheushostert/Repos/solar-neutrino-visible-decays/source/flavour_transitions.py:36: RuntimeWarning: inva

In [71]:
def plot_for_USQR_compare_spline(uesqr=0.01, umusqr=0.1, utausqr=0.00):
    fsize=14
    rc('text', usetex=True)
    rcparams={'axes.labelsize':9,'xtick.labelsize':fsize,'ytick.labelsize':fsize,\
                    'figure.figsize':(1.2*3.7,6)	}
    rc('font',**{'family':'serif', 'serif': ['computer modern roman']})
    matplotlib.rcParams['hatch.linewidth'] = 0.1  # previous pdf hatch linewidth

    rcParams.update(rcparams)
    axes_form1  = [0.16,0.13+0.46,0.82,0.35]
    axes_form2  = [0.16,0.11,0.82,0.35]
    fig = plt.figure()
    ax = fig.add_axes(axes_form1)
    ax2 = fig.add_axes(axes_form2)


    E = np.logspace(np.log10(0.1),np.log10(50),100)

    USQR = [uesqr, umusqr, utausqr]
    # neutrinos
    ax.text(30,1.05,r'$\nu$',fontsize=15)
    ax.set_title(fr'$|U_{{e4}}|^2={uesqr}, |U_{{\mu4}}|^2={umusqr}, |U_{{\tau4}}|^2={utausqr}$', fontsize=14)
    ax.plot(E, osc.Pse_avg(E, nue_to_nue, USQR=USQR, approx=False), lw=1.0, color='black', dashes=(6,0), label=r'$\left\langle P_{\hat{\nu}_{s}\to \nu_e}\right\rangle_N$')
    ax.plot(E, osc.Pse_spline_nu(E, uesqr, umusqr), lw=1.0, color='red', dashes=(6,0), label=r'$\left\langle P_{\hat{\nu}_{s}\to \nu_e}\right\rangle_N$ approx')
  
    # antineutrinos
    ax2.text(30,1.05,r'$\overline{\nu}$',fontsize=15)
    ax2.plot(E, osc.Pse_avg(E, -nue_to_nue, USQR=USQR, approx=False), lw=1.0, color='black', dashes=(6,0), label=r'$\left\langle P_{\overline{\hat{\nu}_{s}}\to \overline{\nu_e}}\right\rangle_N$')
    ax2.plot(E, osc.Pse_spline_nubar(E, uesqr, umusqr), lw=1.0, color='red', dashes=(6,0), label=r'$\left\langle P_{\overline{\hat{\nu}_{s}}\to \overline{\nu_e}}\right\rangle_N$ approx')
 
    ##############
    # STYLE
    # ax.set_yscale('log')
    ax.legend(loc='upper left',frameon=False,ncol=2,fontsize=12,handletextpad=0.3)
    ax2.legend(loc='upper left',frameon=False,ncol=2,fontsize=12,handletextpad=0.3)

    ax.set_ylim(0,1.2)
    ax2.set_ylim(0,1.2)

    ax.set_xlim(0.1,50)
    ax2.set_xlim(0.1,50)

    ax.set_xscale('log')
    ax2.set_xscale('log')

    ax.set_ylabel(r'Probability',fontsize=15)
    ax2.set_ylabel(r'Probability',fontsize=15)

    ax.set_xlabel(r'$E_\nu/$MeV',fontsize=15)
    ax2.set_xlabel(r'$E_\nu/$MeV',fontsize=15)

    fig.savefig(f'plots/Psolar_spline_usqr_{uesqr:.1g}_{umusqr:.1g}_{utausqr:.1g}.pdf')

In [72]:
plot_for_USQR_compare_spline(uesqr=0.01, umusqr=0.00, utausqr=0.00)

/Users/matheushostert/Repos/solar-neutrino-visible-decays/source/flavour_transitions.py:36: RuntimeWarning: invalid value encountered in true_divide
  rx = ma.masked_array(data=-VNC/VCC, mask= ~(VCC != 0), fill_value=0.0)
/Users/matheushostert/Repos/solar-neutrino-visible-decays/source/flavour_transitions.py:36: RuntimeWarning: invalid value encountered in true_divide
  rx = ma.masked_array(data=-VNC/VCC, mask= ~(VCC != 0), fill_value=0.0)
